<a href="https://colab.research.google.com/github/raulrocha/reporta-ai-prototipo/blob/main/Reporta_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai

In [ ]:
!pip install -q google-adk

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
from IPython.display import HTML, Markdown

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai
from google.genai import types
from IPython.display import HTML, Markdown

cliente = genai.Client()

modelo = "gemini-2.0-flash"


In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
# Essa função não está sendo usada.
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def agente_classificacao(imagem, descricao):

    classificacao=Agent(
        name = "agente_classificacao",
        model = modelo,
        description =  "Agente que analisa imagens e classifica-as",
        tools = [google_search],
        instruction = """Você é um assistente de classificação de um aplicativo que tem como objetivo identificar tipos de problemas urbanos, como buraco, vazamento de água, acidentes nas vias, postes caídos, etc.
        Sua função é receber imagens ou links de imagens web e uma descrição sobre ela, analisar e baseado no que você conseguir identificar, terá que classificar se a imagem se trata de um alguns dos problemas
        urbanos citados. Em seguida você deve dizer qual Órgão ou empresa é responsável pela resolução desse problema.
        Exemplos: Se você identificar um poste caído, o responsável é a Companhia Elétrica,
        se identificar um buraco na rua ou na calçada, o responsável será a Prefeitura,
        se identificar um vazamento de água, a Companhia de abastecimento de água será a responsável,
        se identificar um acidente, o SAMU será o responsável.
        As respostas deverão ser sempre em duas linhas:
        Problema: seguido do tipo de problema
        Responsável: seguido do nome do órgão/empresa responsável
        Exemplo de resposta:

        Problema: Vazamento de água
        Responsável: Companhia de abastecimento de água

        Use a descrição fornecida como contexto da imagem.
        Você pode ver um buraco com água e achar que é um vazamento, mas a água pode ser da chuva, por exemplo. Essa observação pode vir na descrição.
        """
    )

    entrada_do_agente_classificacao = f"""
    Imagem: {imagem}
    Descrição: {descricao}
    """
    resposta_do_agente_classificacao = call_agent(classificacao, entrada_do_agente_classificacao)
    return resposta_do_agente_classificacao

In [ ]:
def agente_buscador(endereco, responsavel):

    buscador=Agent(
        name = "agente_buscador",
        model = modelo,
        description = "Agente que buscar informações sobre email e telefone de órgãos prestadores de serviços públicos que possivelmente possam sanar os problemas publicados pelo Reporta AÍ",
        tools = [google_search],
        instruction = """Você é um assistente de pesquisa.
        A sua tarefa é usar a ferramenta de busca do google (google_search) para pesquisar contatos de email e telefone de órgãos públicos ou privados.
        Esses órgãos são responstáveis por prestarem serviços públicos.
        Sua pesquisa deverá ser baseada em 2 parâmetros que irá receber
        1 parâmetro: Responsável(que geralmente será uma prefeitura, o SAMU ou empresas fornecedoras de água e energia, por exemplo).
        2 parâmetro: Endereço(Geralmente será composto por Rua, bairro e/ou cidade ou apenas as coordenadas geográficas).
        A busca deverá ser baseada nos parâmetros recebidos.
        Como resposta, forneça no máximo 1 contatos de cada(1 email e 1 telefone) e o nome da cidade.
        Nas buscas, dê preferência para publicações mais recentes, para evitar respostas falsas/desatualizadas.
        Use a ferramenta google_search apenas para pesquisar contatos.
        Exemplo de parâmetros:
        Responsável: Prefeitura
        Localização: R. Imperial, 203 - São José, Recife - PE
        Baseado nos parâmetros, você deve pesquisar sobre o contato da Prefeitura da Cidade de Recife.
        As respostas deverão ser sempre em 3 linhas:
        Email:
        Telefone:
        Cidade:
        Exemplo de resposta:

        Email: ouvidoria@recife.pe.gov.b
        Telefone: 813355-2950
        Cidade: Recife

        Se não encontrar algum dos contatos, responda exatamente com a palavra 'indisponível', sem aspas, tudo minúsculo, no campo correspondente
        Exemplo de resposta quando não encontrar o contato email:
        Email: indisponível

        Porém, faça uma pesquisa mais aprofundada e possivelmente encontrará emails e telefones válidos.

        """
    )

    entrada_do_agente_buscador = f"""
    Responsável: {responsavel}
    Localização: {endereco}
    """
    resposta_do_agente_buscador = call_agent(buscador, entrada_do_agente_buscador)
    return resposta_do_agente_buscador

In [ ]:
def agente_final_de_rerpote(tipo_problema, descricao, localizacao, imagem_url, email, telefone):

    reporte_final = Agent(
        name = "agente_final_de_rerpote",
        model = modelo,
        description = "Agente que finaliza o relatório para enviar por email",
        instruction = """Você é um assistente especializado no envio de email.
        Sua tarefa é enviar um email para um destinatário de acordo com o modelo padrão, substituindo os dados relevantes de acordo com os parâmetros recebidos.

        _________________________________________________________________

        Exemplo dos dados recebidos:
        Tipo de Problema: (tipo_problema)
        Descrição: (descricao)
        Localização: (localizacao)
        Imagem: (imagem_url)
        email: (email)
        telefone: (telefone)

        Modelo do email:

        Assunto: Reporte de Problema Urbano - (tipo_problema)
        Destinatário: (destinatario)
        Telefone: (telefone)

        Prezados,

        Este email é para reportar um problema urbano identificado:

        Tipo de Problema: (tipo_problema)
        Descrição: (descricao)
        Localização: (localizacao)
        Link da Imagem: (imagem_url)

        Informações de Contato do Reportante: opcional

        Por favor, tomem as providências necessárias para solucionar este problema o mais breve possível.

        Atenciosamente,
        Equipe Reporta Aí (Simulação)

        _________________________________________________________________

        """
    )

    entrada_do_agente_final_de_rerpote = f"""
    Tipo de Problema: {tipo_problema}
    Descrição: {descricao}
    Localização: {localizacao}
    Imagem: {imagem_url}
    email: {email}
    telefone: {telefone}
    """

    resposta_do_agente_final = call_agent(reporte_final, entrada_do_agente_final_de_rerpote)
    return resposta_do_agente_final


In [ ]:
def agente_redator_redes_sociais(imagem, responsavel, cidade):

    redator = Agent(
        name = "agente_redator_redes_sociais",
        model = modelo,
        description = "Agente criador de conteúdo para redes sociais",
        tools = [google_search],
        instruction = """Você é um redator especializado em criação de conteúdo focado em publicações para o Instagram.
        Sua tarefa é analisar uma imagem/link e gerar um texto em tom crítico social sobre problemas urbanos(baseado na imagem), exemplos: buracos nas vias ou vazamentos de água.
        A postagem deve ser direcionada ao responsável pelo problema identificado para que o mesmo se prontifique em resolver o problema.
        Use o google_search para pesquisar o @do_responsavel no instagram para que o mesmo seja marcado na publicação.
        O texto deve ser escrito em português brasileiro. Use texto formal e educado.
        Incluir de 2 a 4 hashtags no final.
        Exemplo de entrada:
        Imagem: link.html(imagem de um buraco na rua)
        Responsável: Prefeitura
        Cidade: São Paulo

        Sua reposta já deve ser exatamente a postagem, sem texto adicional.
        """
    )

    entrada_do_agente_redator = f"""
    Imagem: {imagem}
    Responsável: {responsavel}
    Cidade: {cidade}
    """
    resposta_do_agente_redator = call_agent(redator, entrada_do_agente_redator)
    return resposta_do_agente_redator

In [ ]:
# Exemplos estáticos de problemas (podemos usar links de imagens públicas)
exemplos_staticos = {
    "1": {
        "imagem": "https://conteudo.imguol.com.br/c/noticias/a7/2017/04/26/26abr2017---buraco-com-identificacao-improvisada-no-meio-da-rua-afonso-schmidt-altura-do-numero-102-em-santana-na-zona-norte-de-sao-paulo-1493254616548_1920x1277.jpg",
        "descricao": "Um buraco grande e perigoso no meio da rua, dificultando o tráfego de veículos e pedestres.",
        "localizacao": "Av. Santo Antônio, 126 - Centro, Garanhuns - PE"
    },
    "2": {
        "imagem": "https://s2-oglobo.glbimg.com/m796L0hm0CeMbxJcD6z8Y2hkP5s=/0x0:1170x1085/888x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_da025474c0c44edd99332dddb09cabe8/internal_photos/bs/2025/c/M/NUIvxrSJ2tN6xeENsigg/img-3995.jpg",
        "descricao": "Um vazamento de água na calçada, com grande desperdício e risco de escorregões.",
        "localizacao": "Rua João Bispo, 31, Centro, Pitimbu - PB"
    },
    "3": {
        "imagem": "https://www.douradosnews.com.br/dourados/poste-derrubado-apos-roubo-de-veiculos-continua-caido-na-marcelino/1085432/",
        "descricao": "Um poste de energia caído bloqueando a via.",
        "localizacao": "R. Afonso Cavalcanti, 455, Rio de Janeiro, Rio de Janeiro"
    },
    "4": {
        "imagem": "https://portalmorada.com.br/wp-content/uploads/2022/01/vitima-de-atropelamento-morre-na-santa-casa-de-araraquara-M0MD.jpg",
        "descricao": "Uma vítima de acidente de atropelamento.",
        "localizacao": "São Paulo - SP"
    }
}

In [ ]:
def extrair_tipo_problema(texto_resposta):
    linhas = texto_resposta.split('\n')
    for linha in linhas:
        if "Problema:" in linha:
            partes = linha.split(':')
            if len(partes) > 1:
                return partes[1].strip().lstrip('*').strip()
    return None

def extrair_responsavel(texto_resposta):
    linhas = texto_resposta.split('\n')
    for linha in linhas:
        if "Responsável:" in linha:
            partes = linha.split(':')
            if len(partes) > 1:
                return partes[1].strip().lstrip('*').strip()
    return None

def extrair_cidade(texto_resposta):
    linhas = texto_resposta.split('\n')
    for linha in linhas:
        if "Cidade:" in linha:
            partes = linha.split(':')
            if len(partes) > 1:
                return partes[1].strip().lstrip('*').strip()
    return None

def extrair_email(texto_resposta):
    linhas = texto_resposta.split('\n')
    for linha in linhas:
        if "Email:" in linha:
            partes = linha.split(':')
            if len(partes) > 1:
                return partes[1].strip().lstrip('*').strip()
    return None

def extrair_telefone(texto_resposta):
    linhas = texto_resposta.split('\n')
    for linha in linhas:
        if "Telefone:" in linha:
            partes = linha.split(':')
            if len(partes) > 1:
                return partes[1].strip().lstrip('*').strip()
    return None

In [ ]:
def processamento_agentes(imagem, descricao, localizacao):

    classificacao = agente_classificacao(imagem, descricao)
    tipo_problema = extrair_tipo_problema(classificacao)
    responsavel = extrair_responsavel(classificacao)
    busca = agente_buscador(localizacao, responsavel)
    cidade = extrair_cidade(busca)
    telefone = extrair_telefone(busca)
    email = extrair_email(busca)

    if not email == "indisponível":
        print(f"Email: {email}")
        if not telefone == "indisponível":
            print(f"Telefone: {telefone}")
        opcao = input("\n\nPara enviar o email digite 1\nPara sair digite outra tecla\n")
        if opcao == "1":
            email_enviado = agente_final_de_rerpote(tipo_problema, descricao, localizacao, imagem, email, telefone)
            print("\n")
            print(email_enviado)
            print("\n")
        else:
            print("\nO email não foi enviado.")
    elif not telefone == "indisponível":
        print(f"Não conseguimos encontrar o email do Responsável, porém se desejar você ligar para o telefone: {telefone}")
    else:
        print("Infelizmente não conseguimos encontrar o email nem o telefone do possível responsável.")

    opcao = input("\nGostaria de um texto personalizado para postar no Instagram sobre esse problema?\nDigite 1 para gerar o texto ou para sair digite outra tecla ")
    if opcao == "1":
        texto = agente_redator_redes_sociais(imagem, responsavel,cidade)
        print(f"\n {texto}")

while True:
    print("\nEscolha uma opção para testar:")
    print("1 - Usar exemplo estático de Buraco")
    print("2 - Usar exemplo estático de Vazamento")
    print("3 - Usar exemplo estático de Poste Caído")
    print("4 - Usar exemplo estático de Acidente de Trânsito")
    print("5 - Fornecer seus próprios dados")
    print("6 - Conversar com o Gemini Flash 2.0")
    print("7 - Sair")

    opcao = input("Digite o número da sua escolha: ")

    if opcao in exemplos_staticos:
        problema = exemplos_staticos[opcao]
        print("\n--- Resultado ---\n")
        print(f"Informações já definidas:\n")
        print(f"Imagem: {problema['imagem']}")
        print(f"Descrição: {problema['descricao']}")
        print(f"Localização: {problema['localizacao']}")
        processamento_agentes(problema["imagem"], problema["descricao"], problema["localizacao"])
    elif opcao == "5":
        print("\nForneça os detalhes do problema:")
        imagem_usuario = input("Link da imagem: ")
        descricao_usuario = input("Descrição do problema: ")
        localizacao_usuario = input("Localização do problema: ")
        print("\n--- Resultado ---\n")
        processamento_agentes(imagem_usuario, descricao_usuario, localizacao_usuario)
    elif opcao == "6":
        chat = cliente.chats.create(model = modelo)
        chat.get_history()
        print("\nVocê agora está interagindo com o Gemini")
        print("\nPara finalizar o chat, é só digitar 'sair'.")
        prompt = input("Esperando prompt: ")
        while prompt != "sair":
            resposta = chat.send_message(prompt)
            print("\nResposta: ", resposta.text)
            print("\nPara finalizar o chat, é só digitar 'sair'.")
            prompt = input("Esperando prompt: ")
    elif opcao == "7":
        print("Encerrando a interação.")
        break
    else:
        print("Opção inválida. Tente novamente escolhendo uma opção válida.")

In [ ]:
%%html
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Reporta Aí</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css">
    <style>
        /* Estilos Globais */
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
        }

        .app-container {
            width: 360px;
            height: 740px;
            background-color: #ffffff;
            border-radius: 20px;
            box-shadow: 0 0 20px rgba(0, 0, 0, 0.1);
            overflow: hidden;
            display: flex;
            flex-direction: column;
            position: relative;
        }

        .screen {
            padding: 20px;
            box-sizing: border-box;
            width: 100%;
            height: 100%;
            display: none; /* Oculto por padrão, JS controla a visibilidade */
            flex-direction: column;
            align-items: center;
            text-align: left;
            overflow-y: auto; /* Permite rolagem */
        }

        /* --- Tela Inicial --- */
        #tela-inicial {
            justify-content: center;
            text-align: center;
            display: flex; /* Para o flex-direction: column funcionar */
        }
        #tela-inicial .logo-placeholder {
            width: 150px; height: 150px;
            background: linear-gradient(135deg, #00c6ff, #0072ff);
            border-radius: 10px; display: flex; justify-content: center; align-items: center; margin-bottom: 30px;
        }
        #tela-inicial .logo-placeholder svg { width: 80px; height: 80px; fill: white; }
        #tela-inicial h1 { font-size: 2.5em; color: #333; margin-bottom: 10px; font-weight: bold; }
        #tela-inicial h2 { font-size: 1.2em; color: #555; margin-bottom: 40px; font-weight: normal; }

        /* --- Botões --- */
        .btn { /* Classe base para botões */
            padding: 15px 20px;
            font-size: 1.1em;
            border-radius: 8px;
            cursor: pointer;
            text-decoration: none;
            transition: background-color 0.3s ease, color 0.3s ease, border-color 0.3s ease;
            width: 100%; /* Ocupa a largura total do container do botão */
            max-width: 320px; /* Limita a largura máxima */
            text-align: center;
            box-sizing: border-box;
            margin-top: 10px;
            margin-bottom: 10px;
        }

        .btn-principal { /* Botão da tela inicial */
            background-color: #ffffff; color: #007bff; border: 2px solid #007bff;
            font-weight: bold; font-size: 1.2em; border-radius: 30px;
        }
        .btn-principal:hover { background-color: #007bff; color: #ffffff; }

        .btn-secundario { /* Botão "Buscar localização" */
            background-color: #87CEEB; color: #000000; border: 2px solid #87CEEB;
            font-weight: 500;
        }
        .btn-secundario:hover { background-color: #76b6d2; }

        .btn-acao-principal { /* Botões verdes: "Classificar", "Contactar" */
            background-color: #28a745; color: #ffffff; border: 2px solid #28a745;
            font-weight: bold; font-size: 1.2em;
        }
        .btn-acao-principal:hover { background-color: #218838; border-color: #1e7e34; }

        /* --- Tela de Reporte --- */
        #tela-reporte { display: none; }
        #tela-reporte .image-upload-area {
            width: 100%; height: 180px; background-color: #e9ecef;
            border: 2px dashed #ced4da; border-radius: 10px;
            display: flex; flex-direction: column; justify-content: center; align-items: center;
            text-align: center; color: #6c757d; margin-bottom: 20px; position: relative; font-size: 0.9em;
        }
        #tela-reporte .image-upload-area .camera-icon-container {
            position: absolute; bottom: 10px; right: 10px; background-color: #dc3545;
            border-radius: 50%; padding: 8px; display: flex; justify-content: center; align-items: center;
            box-shadow: 0 2px 4px rgba(0,0,0,0.2);
        }
        #tela-reporte .image-upload-area .camera-icon-container i { color: white; font-size: 1.2em; }
        #tela-reporte .image-upload-area .camera-icon-container .fa-plus {
            font-size: 0.6em; position: absolute; top: 1px; right: 2px; color: white;
            background-color: #dc3545; border-radius: 50%; padding: 1px;
        }
        #tela-reporte .form-group { width: 100%; margin-bottom: 15px; }
        #tela-reporte .form-group label { display: block; margin-bottom: 5px; font-weight: bold; color: #333; }
        #tela-reporte .form-group input[type="text"],
        #tela-reporte .form-group textarea {
            width: 100%; padding: 12px; border: 1px solid #ced4da;
            border-radius: 8px; box-sizing: border-box; font-size: 1em;
        }
        #tela-reporte .form-group textarea { height: 80px; resize: vertical; }

        /* --- Tela de Resumo --- */
        #tela-resumo { display: none; }
        #tela-resumo .problem-image {
            width: 100%;
            max-width: 320px; /* Limita a largura da imagem */
            height: auto; /* Mantém a proporção */
            max-height: 200px; /* Limita altura máxima */
            object-fit: cover; /* Garante que a imagem cubra a área sem distorcer */
            border-radius: 10px;
            margin-bottom: 20px;
            background-color: #e0e0e0; /* Cor de fundo caso a imagem não carregue */
        }
        #tela-resumo .info-group { width: 100%; margin-bottom: 15px; }
        #tela-resumo .info-group strong { display: block; margin-bottom: 5px; font-size: 1.1em; color: #333; }
        #tela-resumo .info-group p {
            background-color: #f8f9fa; padding: 10px; border-radius: 8px;
            border: 1px solid #e9ecef; margin: 0; font-size: 1em; color: #555;
        }
        #tela-resumo .info-group.location p { /* Estilo específico para localização */
            background-color: #cce5ff; /* Azul claro, similar à imagem */
            color: #004085; /* Texto azul escuro */
            border-color: #b8daff;
        }

        /* --- Tela de Enviado --- */
        #tela-enviado { display: none; }
        #tela-enviado .success-message {
            background-color: #28a745; color: white; padding: 15px;
            border-radius: 8px; text-align: center; font-size: 1.3em;
            font-weight: bold; margin-bottom: 20px; width: 100%;
        }
        #tela-enviado .report-details { width: 100%; font-size: 0.95em; }
        #tela-enviado .report-details strong { color: #333; }
        #tela-enviado .report-details p { margin: 5px 0 15px 0; color: #555; line-height: 1.5; }
        #tela-enviado .report-image {
            width: 100%;
            max-width: 280px; /* Imagem um pouco menor na tela de enviado */
            height: auto;
            max-height: 180px;
            object-fit: cover;
            border-radius: 8px;
            margin: 10px auto 15px auto; /* Centraliza a imagem */
            display: block;
            background-color: #e0e0e0;
        }
        #tela-enviado .contact-info p {
            background-color: #f8f9fa; padding: 8px; border-radius: 5px;
            border: 1px solid #e9ecef; font-size: 0.9em;
        }
    </style>
</head>
<body>
    <div class="app-container">
        <div id="tela-inicial" class="screen">
            <div class="logo-placeholder">
                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm0 18c-4.41 0-8-3.59-8-8s3.59-8 8-8 8 3.59 8 8-3.59 8-8 8zm-1-13h2v6h-2zm0 8h2v2h-2z"/></svg>
            </div>
            <h1>Reporte Aí</h1>
            <h2>Sua ferramenta cidadã</h2>
            <button id="btn-interagir-gemini" class="btn btn-principal">Interagir com o Gemini (via Colab)</button>
            <button id="btn-enviar-reporte" class="btn btn-principal" style="margin-top: 20px;">Enviar Reporte</button>
        </div>

        <div id="tela-reporte" class="screen">
            <div class="image-upload-area">
                Imagem/foto <br> do problema
                <div class="camera-icon-container">
                    <i class="fas fa-camera"></i><i class="fas fa-plus"></i>
                </div>
            </div>
            <div class="form-group">
                <label for="descricao">Descrição:</label>
                <textarea id="descricao" name="descricao" placeholder="Descreva o problema aqui..."></textarea>
            </div>
            <div class="form-group">
                <label for="localizacao">Localização:</label>
                <input type="text" id="localizacao" name="localizacao" placeholder="Ex: Rua das Palmeiras, 123">
            </div>
            <button class="btn btn-secundario">Buscar localização</button>
            <button id="btn-classificar-problema" class="btn btn-acao-principal">Classificar problema</button>
        </div>

        <div id="tela-resumo" class="screen">
            <img id="resumo-imagem" src="http://selesnafes.com/wp-content/uploads/2018/06/cano-quebrado.jpg"
                 alt="Imagem do Problema Reportado"
                 class="problem-image"
                 onerror="this.onerror=null; this.src='https://placehold.co/320x180/e0e0e0/757575?text=Imagem+Indisponível';">

            <div class="info-group">
                <strong>Descrição:</strong>
                <p id="resumo-descricao">Tubulação estourada em via pública</p>
            </div>
            <div class="info-group">
                <strong>Responsável:</strong>
                <p id="resumo-responsavel">Companhia de Água e Esgoto do Amapá</p>
            </div>
            <div class="info-group location">
                <strong>Localização:</strong>
                <p id="resumo-localizacao">Canal do Jandiá, Zona Norte, Macapá - AP</p>
            </div>
            <button id="btn-contactar-responsaveis" class="btn btn-acao-principal">Contactar Responsáveis</button>
        </div>

        <div id="tela-enviado" class="screen">
            <div class="success-message">
                <i class="fas fa-check-circle"></i> Parabéns! <br>Seu reporte foi enviado!
            </div>
            <div class="report-details">
                <p><strong>Assunto:</strong> Reporte de Problema Urbano - <span id="enviado-assunto"></span></p>
                <p>Prezados,</p>
                <p>Este email é para reportar o problema a seguir:</p>
                <p><strong>Tipo de Problema:</strong> <span id="enviado-tipo-problema"></span><br>
                    <strong>Descrição:</strong> <span id="enviado-descricao"></span><br>
                    <strong>Localização:</strong> <span id="enviado-localizacao"></span>
                </p>
                <p><strong>Imagem:</strong></p>
                <img id="enviado-imagem" src="http://selesnafes.com/wp-content/uploads/2018/06/cano-quebrado.jpg"
                     alt="Imagem do Problema Enviado"
                     class="report-image"
                     onerror="this.onerror=null; this.src='https://placehold.co/280x180/e0e0e0/757575?text=Imagem+Indisponível';">

                <div class="contact-info">
                    <p><strong>Informações de Contato do Reportante:</strong> Opcional</p>
                    <p><strong>Informações de Contato do Possível Responsável:</strong><br>
                        Email: <span id="enviado-email">contato@caesa.ap.gov.br</span><br>
                        Telefone: <span id="enviado-telefone">0800 086 0116</span>
                    </p>
                </div>
                <p>Por favor, tomem as providências necessárias para solucionar este problema o mais breve possível.</p>
                <p>Atenciosamente,<br>Equipe Reporta Aí</p>
            </div>
        </div>
    </div>

    <script>
        // Seleciona os elementos das telas
        const telaInicial = document.getElementById('tela-inicial');
        const telaReporte = document.getElementById('tela-reporte');
        const telaResumo = document.getElementById('tela-resumo');
        const telaEnviado = document.getElementById('tela-enviado');

        // Seleciona os botões de navegação
        const btnInteragirGemini = document.getElementById('btn-interagir-gemini');
        const btnEnviarReporte = document.getElementById('btn-enviar-reporte');
        const btnClassificarProblema = document.getElementById('btn-classificar-problema');
        const btnContactarResponsaveis = document.getElementById('btn-contactar-responsaveis');

        // Seleciona os campos de entrada da Tela de Reporte
        const descricaoInput = document.getElementById('descricao');
        const localizacaoInput = document.getElementById('localizacao');

        // Seleciona os elementos de exibição da Tela de Resumo
        const resumoDescricao = document.getElementById('resumo-descricao');
        const resumoLocalizacao = document.getElementById('resumo-localizacao');
        const resumoResponsavel = document.getElementById('resumo-responsavel');
        const resumoImagem = document.getElementById('resumo-imagem'); // Se você quiser transferir a imagem (simulada)

        // Seleciona os elementos de exibição da Tela de Enviado
        const enviadoAssunto = document.getElementById('enviado-assunto');
        const enviadoTipoProblema = document.getElementById('enviado-tipo-problema');
        const enviadoDescricao = document.getElementById('enviado-descricao');
        const enviadoLocalizacao = document.getElementById('enviado-localizacao');
        const enviadoEmail = document.getElementById('enviado-email');
        const enviadoTelefone = document.getElementById('enviado-telefone');
        const enviadoImagem = document.getElementById('enviado-imagem'); // Se você quiser transferir a imagem (simulada)

        // Array com todas as telas para facilitar o gerenciamento
        const todasAsTelas = [telaInicial, telaReporte, telaResumo, telaEnviado];

        // Objeto simulando o dicionário de contatos (em JavaScript)
        const contatosSimuladosJS = {
            "Companhia de Água e Esgoto do Amapá": {
                "email": "contato@caesa.ap.gov.br",
                "telefone": "0800 086 0116"
            },
            "Prefeitura": {
                "email": "ouvidoria@macapa.ap.gov.br", // Exemplo
                "telefone": "Não disponível"
            },
            // Adicione mais responsáveis e contatos conforme necessário
        };

        // Função para mostrar uma tela e esconder as outras
        function mostrarTela(telaParaMostrar) {
            todasAsTelas.forEach(tela => {
                if (tela) { // Verifica se o elemento da tela existe
                    tela.style.display = 'none';
                }
            });
            if (telaParaMostrar) { // Verifica se a tela a ser mostrada existe
                telaParaMostrar.style.display = 'flex'; // Usamos 'flex'
            }
        }

        // Event Listeners para navegação
        if (btnInteragirGemini) {
            btnInteragirGemini.addEventListener('click', function() {
                alert("A funcionalidade de interação direta com o Gemini está disponível para testes executando as células de código Python no Colab e escolhendo a opção 6.");
                // Ou, se você quiser criar uma tela de chat simulada:
                // mostrarTela(telaChatGemini); // (Você precisaria criar a telaChatGemini no HTML)
            });
        }

        if (btnEnviarReporte) {
            btnEnviarReporte.addEventListener('click', function() {
                mostrarTela(telaReporte);
            });
        }

        if (btnClassificarProblema) {
            btnClassificarProblema.addEventListener('click', function() {
                // Pega os valores da tela de reporte
                const descricaoValue = descricaoInput.value;
                const localizacaoValue = localizacaoInput.value;

                // Simulação da classificação (aqui você chamaria o Gemini em um app real)
                const tipoProblemaSimulado = "Vazamento de água";
                const responsavelSimulado = "Companhia de Água e Esgoto do Amapá";
                const assuntoSimulado = `Reporte de Problema Urbano - ${tipoProblemaSimulado}`;

                // Popula os campos da tela de resumo
                resumoDescricao.textContent = descricaoValue || "Descrição não informada";
                resumoLocalizacao.textContent = localizacaoValue || "Localização não informada";
                resumoResponsavel.textContent = responsavelSimulado;
                // Se você tiver uma lógica para a imagem, pode adicioná-la aqui

                // Prepara os dados para a tela de envio (poderia ser feito no botão de enviar também)
                enviadoAssunto.textContent = assuntoSimulado;
                enviadoTipoProblema.textContent = tipoProblemaSimulado;
                enviadoDescricao.textContent = descricaoValue || "Descrição não informada";
                enviadoLocalizacao.textContent = localizacaoValue || "Localização não informada";

                // Navega para a tela de resumo
                mostrarTela(telaResumo);
            });
        }

        if (btnContactarResponsaveis) {
            btnContactarResponsaveis.addEventListener('click', function() {
                // Pega o responsável da tela de resumo
                const responsavelResumo = resumoResponsavel.textContent;

                // Busca o contato simulado
                const contato = contatosSimuladosJS[responsavelResumo];

                // Preenche os campos da tela de envio com os contatos (se encontrados)
                if (contato) {
                    enviadoEmail.textContent = contato.email || "Não disponível";
                    enviadoTelefone.textContent = contato.telefone || "Não disponível";
                } else {
                    enviadoEmail.textContent = "Contato não encontrado";
                    enviadoTelefone.textContent = "Contato não encontrado";
                }

                // Navega para a tela de enviado
                mostrarTela(telaEnviado);
            });
        }

        // Inicialização: Garante que a tela inicial seja exibida ao carregar
        mostrarTela(telaInicial);

    </script>
</body>
</html>
